In [1]:
import sys , os 
import numpy as np
import math as m
import pandas as pd

import plotly.offline as py

import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

In [2]:
mesurePlot="/home/crct/CRCT/JupyterNotebook/gammaIndex/6E/new/fl6e20.csv"
gatePlot="/home/crct/CRCT/JupyterNotebook/gammaIndex/6E/new/gl6e20.csv"
emcPlot="/home/crct/CRCT/JupyterNotebook/gammaIndex/6E/new/emcl6e20.csv"
gateuncertaityPlot="/home/crct/CRCT/JupyterNotebook/gammaIndex/6E/new/uncertainty/lg6e20_uncertainty.csv"
mesurePlotData=pd.read_csv(mesurePlot, sep=';')
gatePlotData=pd.read_csv(gatePlot, sep=',')
emcPlotData=pd.read_csv(emcPlot, sep=';')

gateuncertaintyPlotData=pd.read_csv(gateuncertaityPlot, sep=',')


In [4]:
LinearInterpolation=True
#Point every (in mm)
#NewStep=0.1

Threshold=True
#Of Dmax
ThresholdValue=0.1

#Gamma Index criterion 
DTA=3 #mm
deltaDmax=0.03 #%


name='Lung6e20'

gammaIndex(mesurePlotData, gatePlotData, emcPlotData, DTA, deltaDmax, name, Threshold)


nombre de points testés : 1276
nombre de points passant le test (<1) : 1027.0
RESULTAT GATE : 
*   Gamma Index Pass Rate 3.0%-3mm: 80.4858934169 %
nombre de points testés : 864
nombre de points passant le test (<1) : 486.0
RESULTAT EMC : 
*   Gamma Index Pass Rate 3.0% - 3mm : 56.25 %


In [3]:
def gammaIndex(mesurePlot, gatePlot, emcPlot, DTA, deltaDmax, name, Threshold=True):

    xGscale=1.9
    xEMCscale=2
    
    #yGscale=1.0
    #yEMCscale=1.0
    

    
    #Create numpy ndarray from pandas dataFrames
    refX=mesurePlotData['pas']
    refY=mesurePlotData['value']
    gateX=gatePlotData['pas']
    gateY=gatePlotData['value']
    emcX=emcPlotData['pas']
    emcY=emcPlotData['value']
    
    gateUx=gateuncertaintyPlotData['pas']
    gateUy=gateuncertaintyPlotData['value']
    
    
    #normalisation au max
    #refY=refY/refY.max()
    #gateY=gateY/gateY.max()
    #emcY=emcY/emcY.max()
    
    refX=refX-(refX.max()/2)
    emcX=emcX-(emcX.max()/2)
    


    # Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
    if LinearInterpolation==True:
        refX2=np.linspace(refX.min(), refX.max(), 2000)
        refY2=np.interp(refX2, refX, refY)
        gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
        gateY2=np.interp(gateX2, gateX, gateY)
        emcX2=np.linspace(emcX.min(), emcX.max(), 2000)
        emcY2=np.interp(emcX2, emcX, emcY)
        
        gateUx2=np.linspace(gateUx.min(), gateX.max(), 2000)
        gateUy2=np.interp(gateUx2, gateUx, gateUy)
        
    
    else:
        refX2= refX
        refY2= refY
        gateX2= gateX
        gateY2= gateY
        emcX2= emcX
        emcY2= emcY
    emcX2=np.flipud(emcX2)
    
    

    
    maskFilm=(refX2 > -20) & (refX2 < -15)
    maskGate=(gateX2 > -20) & (gateX2 < -15)
    maskEmc=(emcX2 > -20) & (emcX2 < -15)

    #normalisation mean homogeneity
    refY2=refY2/refY2[maskFilm].mean()
    gateY2=gateY2/gateY2[maskGate].mean()
    emcY2=emcY2/emcY2[maskEmc].mean()
    gateUy2=gateUy2*gateY2
       
    #normalisation max 
    #refY2=refY2/refY2.max()
    #gateY2=gateY2/gateY2.max()
    #emcY2=emcY2/emcY2.max()
    
    # Scaling for relative comparison
    #gateX2=gateX2+DoseScalingFactor[0]
    #emcX2=emcX2+DoseScalingFactor[1]
    
        #SCALING
    gateX2=gateX2+xGscale
    emcX2=emcX2+xEMCscale
    
    #gateY2=gateY2*0.98
    #emcY2=emcY2*0.97
    #if DoseScaling:        
        #gateY2=gateY2*DoseScalingFactor[2]
        #emcY2=emcY2*DoseScalingFactor[3]
    #SCALING   
    #gateY2=gateY2*yGscale
    #emcY2=emcY2*yEMCscale
        
    #  GATE  #
    
    #create numpy ndarray for gamma comprison
    deltaD=np.zeros(len(refY2))
    gammaIndex=np.zeros(len(gateX2))
    gammaIndexTEMP=np.zeros(len(refX2))
    drTEMP=np.zeros(len(refX2))
    dr=np.zeros(len(refX2))

    #compute Gamma Index 1D
    for i in range(len(gateX2)):
        for j in range(len(refX2)):
            dr[j]=(gateX2[i]-refX2[j])
            deltaD[j]=gateY2[i]-refY2[j]
            gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))           
        gammaIndex[i]=gammaIndexTEMP.min()

    #Compute Gamma Index Pass Rate (with threshold or not)
    nok=0.0
    a=0
    if Threshold==True: 
        for i in range(len(gammaIndex)):

            if gateY2[i]>0.10:
                a=a+1
                if gammaIndex[i]<1:
                    nok=nok+1
        print ('nombre de points testés : ' +str(a))
        print ('nombre de points passant le test (<1) : ' +str(nok))
        passRate=(nok/a)*100
    else:
        nok=(gammaIndex < 1).sum()
        passRate=nok*100.0/len(gammaIndex)
                                                                        
    #print results
    percentDmax=deltaDmax*100
    print('RESULTAT GATE : ')
    print('*   Gamma Index Pass Rate '+str(percentDmax) + '%-' +str(DTA) + 'mm: '  +str(passRate) +' %')
    
    #    EMC   #
    
    #create numpy ndarray for gamma comprison
    deltaD=np.zeros(len(refY2))
    gammaIndex2=np.zeros(len(emcX2))
    gammaIndexTEMP=np.zeros(len(refX2))
    drTEMP=np.zeros(len(refX2))
    dr=np.zeros(len(refX2))

    #compute Gamma Index 1D
    for i in range(len(emcX2)):
        for j in range(len(refX2)):
            dr[j]=(emcX2[i]-refX2[j])
            deltaD[j]=emcY2[i]-refY2[j]
            gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))
        #print gammaIndexTEMP.argmin()
        gammaIndex2[i]=gammaIndexTEMP.min()
    #print gammaIndex2
    #Compute Gamma Index Pass Rate (with threshold or not)
    nok=0.0
    a=0
    if Threshold==True: 
        for i in range(len(gammaIndex2)):

            if emcY2[i]>0.10:
                a=a+1
                if gammaIndex2[i]<1:
                    nok=nok+1
        print ('nombre de points testés : ' +str(a))
        print ('nombre de points passant le test (<1) : ' +str(nok))
        passRate2=(nok/a)*100
    else:
        nok=(gammaIndex2 < 1).sum()
        passRate2=nok*100.0/len(gammaIndex2)

    #print results
    percentDmax=deltaDmax*100
    print('RESULTAT EMC : ')
    print('*   Gamma Index Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm : '  +str(passRate2) +' %')
    
    emcY3=np.flipud(emcY2)
    emcX3=np.flipud(emcX2)
    
    #compute diff % 
    diffGate=((refY2-gateY2)/refY2)*100
    diffEmc=((refY2-emcY3)/refY2)*100
    
    tracetest = go.Scatter(
        x = gateX,
        y = gateY,
        mode = 'lines', 
        name = 'GATE', 
        line = dict(
            color = ('rgba(44, 160, 44, 100)'),
            dash='dot', 
            width=3
        ),
        error_y=dict(
            type='data',
            array=gateUy,
            visible=True,
            thickness=0.5,
            width=1.5,
            opacity=0.3,
            color='#9CEF9C'
        )
    )


    # Plot profil or PDD with plotly      
    trace1 = go.Scatter(
        x = refX2,
        y = refY2,
        mode = 'lines', 
        name = 'Film Measure', 
        line = dict(
            color = ('rgba(214, 39, 40, 100)')
        )
    )

    trace2 = go.Scatter(
        x = gateX2,
        y = gateY2,
        mode = 'lines', 
        name = 'GATE', 
        line = dict(
            color = ('rgba(44, 160, 44, 100)'),
            dash='dot', 
            width=3
        ),
        error_y=dict(
            type='data',
            array=gateUy2,
            visible=True,
            thickness=0.5,
            width=1.5,
            opacity=0.5,
            color='#9CEF9C'
        )
    )

    #Plot gamma Index with plotly
    trace3 = go.Scatter(
        x = emcX2,
        y = emcY2,
        mode = 'lines', 
        name = 'eMC ', 
        line = dict(
            color=('rgba(31, 119, 180, 100)'), 
            dash='dash'
        ),
        error_y=dict(
            type='percent',
            value=2,
            visible=True,
            thickness=0.5,
            width=1.5,
            opacity=0.5, 
            color='#7CBAE6'
        )
    )
    
    
        #Plot gamma Index with plotly
    trace4 = go.Scatter(
        x = gateX2,
        y = gammaIndex,
        mode = 'lines', 
        name = 'Gamma index GATE ', 
         line = dict(
            color = ('rgba(44, 160, 44, 100)'),
            dash='dot', 
            width=3
        ),
    )
     
        
        #Plot gamma Index with plotly
    trace5 = go.Scatter(
        x =emcX2,
        y = gammaIndex2,
        mode = 'lines', 
        name = 'Gamma index eMC ', 
          line = dict(
            color=('rgba(31, 119, 180, 100)'), 
            dash='dot',
            width=3,
        ),
    )

    
    layout1= go.Layout(
        title='Dose Profile - Lung Heterogeneity - 6 MeV',
        titlefont=dict(size=24),
        annotations=[dict(
            x=25,
            y=0.5,
            xref='gateX2',
            yref='gammaIndex',
            showarrow=False, 
            text='Depth : 20 mm',
            font=dict(
            size=18)
         )],
        xaxis=dict(
            range=[-100, 100],
            title='Off distance axis (mm)',
            titlefont=dict(
                size=20
            )
    ),      
        yaxis=dict(
            title='Relative Dose',
            titlefont=dict(
                size=20)
        )
    )
    
    val1=str(round(passRate,2))
    val2=str(round(passRate2,2))
        
    layout2= go.Layout(
        title='Gamma index - Lung Heterogeneity - 6 MeV',
        titlefont=dict(size=24),
        annotations=[
            dict(
                x=-25,
                y=3,
                xref='gateX2',
                yref='gammaIndex',
                showarrow=False, 
                text='GIPR 3%-3mm Gate: ' + val1,
                font=dict(
                size=18)
         ),
            dict(
                x=-25,
                y=3.5,
                xref='gateX2',
                yref='gammaIndex',
                showarrow=False, 
                text='GIPR 3%-3mm eMC: ' + val2,
                font=dict(
                size=18)
         )              
        ],
        xaxis=dict(
            title='Off distance axis (mm)',
            range=[-60, 60],
            titlefont=dict(
                    size=20)
        ),
        yaxis=dict(
            title='Gamma index value',
            range=[0, 5],
            titlefont=dict(
                    size=20)
        )
        
    )
    
    
    trace6 = go.Scatter(
        x = gateX2,
        y = diffGate,
        mode = 'lines', 
        name = 'Difference % GATE ', 
        line = dict(
            color=('rgba(255, 127, 17, 100)'), 
            dash='dash'
        )
    )
         
    trace7 = go.Scatter(
        x = emcX2,
        y = diffEmc,
        mode = 'lines', 
        name = 'Difference % EMC ', 
        line = dict(
            color=('rgba(23, 190, 207, 100)'), 
            dash='dash'
        )
    )

    data1 = [trace1, trace2, trace3]
    fig1= go.Figure(data=data1, layout=layout1)   
    py.iplot(fig1, filename=name+'_plot', image='png') 
    
    data2= [trace4, trace5]
    fig2= go.Figure(data=data2, layout=layout2)
    py.iplot(fig2, filename=name+'_gamma', image='png') 
    
                              
    return;

